In [54]:
from dolfin import *
from PIL import Image, ImageOps
import numpy as np
from numpy import asarray
import ufl

In [55]:
img_w = .1 # meters
img_h = .1 # meters
area_total = img_w * img_h

In [56]:
image = Image.open('cat_posing.jpg')
#image = Image.open('mehul.png')
im2 = ImageOps.grayscale(image)
brightness = asarray(im2)
total_brightness = sum(sum(brightness))

In [57]:
imesh = RectangleMesh.create([Point(0, 0), Point(img_w, img_h)],[512,512],CellType.Type.quadrilateral)
cmesh = RectangleMesh.create([Point(0, 0), Point(img_w, img_h)],[512,512],CellType.Type.quadrilateral)

In [58]:
for ii in range(3):
    # Build function space with Lagrange multiplier
    P1 = FiniteElement("Lagrange", cmesh.ufl_cell(), 1)
    R = FiniteElement("Real", cmesh.ufl_cell(), 0)
    W = FunctionSpace(cmesh, P1 * R)

    # Define variational problem
    (u, c) = TrialFunction(W)
    (v, d) = TestFunctions(W)

    D = FunctionSpace(cmesh,"DG",0)
    br = Function(D)
    br.vector().vec().array[:] = brightness.reshape(cmesh.num_cells())

    V = FunctionSpace(cmesh,"CG",1)
    br= project(br,V)

    (cr,dr) = Function(W).split()
    (ar,er) = Function(W).split()

    cr.vector().vec().array[0:br.vector().size()] = br.vector()[:]

    area = Function(D)
    vv = TestFunction(D)

    area.vector()[:] = assemble(1*vv*dx)
    ar.vector().vec().array[0:area.vector().size()] = area.vector()[:]

    loss = cr / Constant(total_brightness) - ar / Constant(area_total)

    a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
    L = -loss*v*dx

    # Compute solution
    w = Function(W)
    solve(a == L, w)
    (u, c) = w.split()

    with XDMFFile("phi.xdmf") as xdmf:
        xdmf.write(u,ii)

    print(norm(project(loss,D)))

    grad_u = project(grad(u),VectorFunctionSpace(cmesh,"CG",1))
    g_u = grad_u.compute_vertex_values()
    with XDMFFile("grad_u.xdmf") as xdmf:
        xdmf.write(grad_u)

    mesh_co=cmesh.coordinates()
    nv = cmesh.num_vertices()
    g_un=np.zeros((nv,2))
    g_un[:,0]=g_u[0:nv]
    g_un[:,1]=g_u[nv:2*nv]
    step = 50

    cmesh.coordinates()[:]=mesh_co+step*g_un

0.00023978957619565322
0.00024329408356993135
0.00024677366963582


In [59]:
width, height = 512, 512

H = 0.2
metersPerPixel = img_w / width
print(metersPerPixel)

n2 = 1
n1 = 1.49

0.0001953125


In [60]:
ZU = FunctionSpace(imesh, "CG", 1)
IU = VectorFunctionSpace(imesh, "CG", 1)
CU = VectorFunctionSpace(cmesh, "CG", 1)

In [61]:
i_coord,c_coord, new_coord = Function(IU), Function(IU), Function(CU)
coord_z = Function(ZU)

In [62]:
coord = Expression(('x[0]','x[1]'),degree = 1) * Constant(metersPerPixel)

i_coord = project(coord,IU)
new_coord = project(coord,CU)

c_coord.vector()[:] = new_coord.vector()[:]

In [63]:
i_coord.vector()[:]

array([1.95312500e-05, 9.04728795e-24, 1.94931030e-05, ...,
       1.95312500e-05, 1.95312500e-05, 1.95312500e-05])

In [64]:
c_coord.vector()[:]

array([1.95303546e-05, 1.00453005e-09, 1.94902046e-05, ...,
       1.95323140e-05, 1.95281671e-05, 1.95323135e-05])

In [65]:
little_h = coord_z * Constant(metersPerPixel)
H_minus_h = H - little_h
dz = project(H_minus_h,ZU)

In [66]:
Nx = project(ufl.tan(ufl.atan((i_coord[0]-c_coord[0])/dz)/(n1-n2)),ZU)
Ny = project(ufl.tan(ufl.atan((i_coord[1]-c_coord[1])/dz)/(n1-n2)),ZU)

In [67]:
Nxy = Function(IU)

In [68]:
Nxy.vector().vec().array[0::2] = Nx.vector()[:]
Nxy.vector().vec().array[1::2] = Ny.vector()[:]

In [69]:
Nxy.vector().min(), Nxy.vector().max()

(-6.522516903347465e-06, 4.543533490995276e-06)

In [70]:
# Build function space with Lagrange multiplier
P1 = FiniteElement("Lagrange", cmesh.ufl_cell(), 1)
R = FiniteElement("Real", cmesh.ufl_cell(), 0)
W = FunctionSpace(cmesh, P1 * R)

# Define variational problem
(u, c) = TrialFunction(W)
(v, d) = TestFunctions(W)

In [71]:
f = Function(W)
f.vector().vec().array[0:-1] = project(div(Nxy),ZU).vector()[:]

In [72]:
f = f.split()[0]

In [73]:
a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
L = inner(f*10000,v)*dx

# Compute solution
w = Function(W)
solve(a == L, w)
(usol, csol) = w.split()

In [74]:
with XDMFFile("heightmap.xdmf") as xdmf:
    xdmf.write(usol)

In [75]:
usol.vector()[:-1].min(),usol.vector().max()

(-0.0018897803858899148, 0.0016312968161748338)

In [76]:
usol.vector()[:]

array([-0.00186678, -0.00186677, -0.00186674, ...,  0.00098882,
        0.00098873, -0.33048122])

In [47]:
i_cord = imesh.coordinates()[:]
c_cord = cmesh.coordinates()[:]

delta = cord-i_cord

dx, dy = delta[:,0]* metersPerPixel,delta[:,1]* metersPerPixel

cord_z = np.zeros(nv)

little_h = cord_z * metersPerPixel
H_minus_h = H - little_h
dz = H_minus_h

Ny = np.tan(np.arctan(dy / dz) / (n1 - n2))
Nx = np.tan(np.arctan(dx / dz) / (n1 - n2))

In [80]:
U_old = VectorFunctionSpace(i_mesh, "CG",1)
U_new = VectorFunctionSpace(mesh_, "CG",1)

In [81]:
old_coord,vectorw_coord = Function(U_old), Function(U_new)

In [82]:
f2 = Expression(('x[0]','x[1]'),degree = 1)

old_coord = project(f2,U_old)
new_coord = project(f2,U_new)

old_coord.vector()[:]=new_coord.vector()[:]

In [86]:
with XDMFFile("old_coord.xdmf") as xdmf:
    xdmf.write(old_coord)

In [ ]:
divergence = zeros(width, height)
# We need to find the divergence of the Vector field described by Nx and Ny

for j = 1:height
    for i = 1:width
        δx = (Nx[i + 1, j] - Nx[i, j])
        δy = (Ny[i, j + 1] - Ny[i, j])
        divergence[i, j] = δx + δy
    end
end
println("Have all the divergences")
println("Divergence sum: $(sum(divergence))")
divergence .-= sum(divergence) / (width * height)

In [ ]:
h = zeros(width, height)
max_update = 0
for i = 1:10000
    max_update = relax!(h, divergence)

    if i % 500 == 0
        println(max_update)
    end
    if max_update < 0.00001
        println("Convergence reached at step $(i) with max_update of $(max_update)")
        break
    end
end

h, metersPerPixel

In [96]:
g_u = grad_u.compute_vertex_values()

In [93]:
g_u[mesh_.num_vertices()]

3.747870501904941e-08

In [100]:
g_un

array([[ 2.64318478e-08,  3.74787050e-08],
       [ 8.36406075e-08,  3.88219996e-08],
       [ 1.82486202e-07,  4.04677279e-08],
       ...,
       [-7.90285747e-07,  4.17947883e-08],
       [-3.65399718e-07,  4.41713315e-08],
       [-1.16803704e-07,  4.73168635e-08]])

In [88]:
mesh_.coordinates()[0,0] = 0.0001

In [89]:
mesh_.coordinates()

array([[0.0001    , 0.        ],
       [0.00019531, 0.0001    ],
       [0.00039063, 0.        ],
       ...,
       [0.09960938, 0.1       ],
       [0.09980469, 0.1       ],
       [0.1       , 0.1       ]])

In [74]:
mesh.num_vertices()

263169

In [44]:
mag = np.sqrt(g_u[1][1]**2+g_u[1][1]**2)

In [45]:
g_u[1]/mag

array([0.46724043, 0.70710678])

In [52]:
mesh.hmin()

0.00016470458559087164

In [2]:
from PIL import Image
import numpy as np
from numpy import asarray

# load the image

image = Image.open('cat_posing.jpg')

# convert image to numpy array

data = asarray(image)

print(type(data))

# summarize shape

print(data.shape)



# create Pillow image

image2 = Image.fromarray(data)

print(type(image2))


# summarize image details

print(image2.mode)

print(image2.size)

<class 'numpy.ndarray'>
(512, 512, 3)
<class 'PIL.Image.Image'>
RGB
(512, 512)


In [3]:
print(data)

[[[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 [[236 224 172]
  [236 224 172]
  [236 224 172]
  ...
  [232 230 189]
  [230 231 189]
  [230 231 189]]

 ...

 [[178 136  78]
  [178 136  78]
  [180 138  80]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[168 126  68]
  [172 130  72]
  [185 143  85]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]

 [[145 100  43]
  [151 109  51]
  [142  99  44]
  ...
  [234 230 192]
  [234 230 192]
  [234 230 192]]]


In [4]:
# Importing Image and ImageOps module from PIL package
from PIL import Image, ImageOps


# applying grayscale method
im2 = ImageOps.grayscale(image)
brightness = asarray(im2)

In [6]:
total_brightness = sum(sum(brightness))

In [7]:
normalized_brightness= np.divide(brightness,total_brightness)

In [9]:
from dolfin import *

In [10]:
mesh = RectangleMesh(Point(0,0), Point(w,h), 512,512)

In [11]:
mesh = RectangleMesh.create([Point(0,0),Point(w,h)],[512,512],CellType.Type.quadrilateral)

In [12]:
V = FunctionSpace(mesh,"CG",1)
D = FunctionSpace(mesh,"DG",0)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [13]:
br = Function(D)

br.vector().vec().array[:] = brightness.reshape(mesh.num_cells())

In [14]:
with XDMFFile("brightness.xdmf") as xdmf:
    xdmf.write(br)

In [15]:
area = Function(D)
v = TestFunction(D)

area.vector()[:] = assemble(1*v*dx)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [16]:
area.vector()[:].sum()

0.009999999999999966

In [17]:
loss = br / Constant(total_brightness) - area / Constant(area_total)

In [18]:
loss = project(loss,V)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [19]:
loss

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 6), FiniteElement('Q', quadrilateral, 1)), 32)

In [20]:
with XDMFFile("loss.xdmf") as xdmf:
    xdmf.write(loss)

In [21]:
# gloss = project(grad(loss),VectorFunctionSpace(mesh,"DG",0))

with XDMFFile("gloss.xdmf") as xdmf:
    xdmf.write(gloss)

In [23]:
ph =  Function(D)

In [24]:
u = TrialFunction(D)
v = TestFunction(D)

a = inner(grad(u), grad(v))*dx

In [25]:
solve(a == -loss*v*dx,ph)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [158]:
with XDMFFile("phi.xdmf") as xdmf:
    xdmf.write(ph)

In [159]:
ph.vector().min()

-inf

In [160]:
ph.vector().max()

-inf